In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer
from qiskit.extensions.standard import CHGate, HGate, CnotGate, CyGate, CzGate
from qiskit.circuit import Measure
from qiskit.circuit.quantumregister import QuantumRegister
from qiskit.circuit.classicalregister import ClassicalRegister
from qiskit import BasicAer
from qiskit.visualization import plot_histogram
from help_fun import cut_edge, generate_sub_circs
import numpy as np
import networkx as nx
import pickle
import argparse
import os

q = QuantumRegister(4, 'q')
c = ClassicalRegister(4, 'c')
circ = QuantumCircuit(q,c)
circ.h([q[2],q[1]])
circ.x([q[2],q[0]])
circ.y(q[3])
circ.cx(q[0], q[1])
circ.cy(q[2], q[3])
circ.ch(q[1], q[2])
circ.x(q[2])
circ.measure(q,c)

for node in circuit_to_dag(circ).op_nodes():
    print(node.data_dict)
    break

{'type': 'op', 'op': <qiskit.extensions.standard.h.HGate object at 0x12e586470>, 'name': 'h', 'qargs': [(QuantumRegister(4, 'q'), 2)], 'cargs': [], 'condition': None}


In [2]:
original_dag = circuit_to_dag(circ)
dag_drawer(original_dag, filename='original_dag.pdf')
circ.draw(output='mpl',filename='original_circ.pdf')

cut_dag = cut_edge(original_dag=original_dag, wire=q[2],source_node_name='cy', dest_node_name='ch')
dag_drawer(cut_dag, filename='cut_dag.pdf')
dag_to_circuit(cut_dag).draw(output='mpl',filename='cut_circ.pdf')

sub_circs, _ = generate_sub_circs(cut_dag, q[2])
for i, sub_circ in enumerate(sub_circs):
    dag_drawer(circuit_to_dag(sub_circ), filename='sub_dag_%d.pdf' % i)
    sub_circ.draw(output='mpl',filename='sub_circ_%d.pdf' % i)

component 0
reg_dict: {'q': QuantumRegister(2, 'q'), 'c': ClassicalRegister(3, 'c'), 'cutQ': QuantumRegister(1, 'cutQ')}
op x [(QuantumRegister(2, 'q'), 0)] []
op h [(QuantumRegister(2, 'q'), 1)] []
op cx [(QuantumRegister(2, 'q'), 0), (QuantumRegister(2, 'q'), 1)] []
op ch [(QuantumRegister(2, 'q'), 1), (QuantumRegister(1, 'cutQ'), 0)] []
op x [(QuantumRegister(1, 'cutQ'), 0)] []
op measure [(QuantumRegister(2, 'q'), 0)] [(ClassicalRegister(3, 'c'), 0)]
op measure [(QuantumRegister(2, 'q'), 1)] [(ClassicalRegister(3, 'c'), 1)]
op measure [(QuantumRegister(1, 'cutQ'), 0)] [(ClassicalRegister(3, 'c'), 2)]
registers in the sub circuit: [QuantumRegister(2, 'q'), QuantumRegister(1, 'cutQ')] [ClassicalRegister(3, 'c')]
total registers counts: {'q': QuantumRegister(2, 'q'), 'c': ClassicalRegister(3, 'c'), 'cutQ': QuantumRegister(1, 'cutQ')}
finished component 0

component 1
reg_dict: {'q': QuantumRegister(2, 'q'), 'c': ClassicalRegister(1, 'c'), 'cutC': ClassicalRegister(1, 'cutC')}
op h [(Q